<a href="https://colab.research.google.com/github/nadavru/iLearn/blob/iLearnML/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import numpy as np
from abc import ABCMeta, abstractmethod
from IPython.display import IFrame
import math
from math import exp, pi
from numpy.linalg import inv

# optimizers

In [45]:
class opt:
    def __init__(self, x, y, lr, momentum): 
      self.x=x
      self.y=y
      self.lr=lr
      self.momentum=momentum

    @abstractmethod
    def step(self, d:list, d2):
      pass

In [46]:
class SGD(opt):
  def __init__(self, x, y, lr, momentum=0): 
      super().__init__(x, y, lr, momentum)
  def step(self, d, d2=None):
    self.x = self.x-self.lr*d[0]
    self.y = self.y-self.lr*d[1]
    return (self.x, self.y)
    

In [47]:
class MomentumSGD(opt):
  def __init__(self, x, y, lr, momentum=0.9): 
      super().__init__(x, y, lr, momentum)
      self.dx, self.dy = 0, 0
  def step(self, d, d2=None):
    self.dx = self.momentum*self.dx - self.lr*d[0]
    self.dy = self.momentum*self.dy - self.lr*d[1]
    self.x = self.x + self.dx
    self.y = self.y + self.dy
    return (self.x, self.y)

In [48]:
class Newton(opt):
  def __init__(self, x, y, lr, momentum=0): 
      super().__init__(x, y, lr, momentum)
  def step(self, d1, d2=None):
    d1 = np.array(d1)
    d2 = np.array(d2)
    d2 = inv(d2)
    d = -np.matmul(d2, d1)
    self.x = self.x + self.lr*d[0]
    self.y = self.y + self.lr*d[1]
    return (self.x, self.y)

# Learning

In [49]:
class Trainer:
    def __init__(self, grid_x, grid_y, lr=0.5, epochs=50, opt=SGD, x=None, y=None): 
        self.lr=lr
        self.x_cur = 2*grid_x*np.random.random_sample()-grid_x if x is None else x
        self.y_cur = 2*grid_y*np.random.random_sample()-grid_y if y is None else y
        self.epochs=epochs

        x = np.linspace(-grid_x, grid_x, 1000)
        y = np.linspace(-grid_y, grid_y, 1000)
        self.X, self.Y = np.meshgrid(x, y)
        self.opt = opt(self.x_cur, self.y_cur, self.lr)

    def __call__(self): 
        return self.X, self.Y, np.where(True, self.calc(self.X, self.Y), 0)
    
    def train(self): 
        self.Xs = [self.x_cur]
        self.Ys = [self.y_cur]
        self.Zs = [self.calc(self.x_cur, self.y_cur)]

        for _ in range(self.epochs):
            d = self.derive(self.x_cur, self.y_cur)
            d2 = self.derive2(self.x_cur, self.y_cur)
            #print(d)
            self.x_cur, self.y_cur = self.opt.step(d, d2)

            self.Xs.append(self.x_cur)
            self.Ys.append(self.y_cur)
            self.Zs.append(self.calc(self.x_cur, self.y_cur))
        return self.Xs, self.Ys, self.Zs

    @abstractmethod
    def display(self): 
      pass
    
    def derive2(self, x, y): 
      return None
    
    @abstractmethod
    def calc(self, x, y): 
      pass

    @abstractmethod
    def derive(self, x, y): 
      pass

In [ ]:
class SVM(Trainer):

  def __init__(self, group_size, p0, p1, grid_x=50, grid_y=50, lr=0.5, epochs=10, opt=SGD): 
        super().__init__(grid_x, grid_y, lr, epochs, opt)
        (x0, y0, var0) = p0
        (x1, y1, var1) = p1
        self.group_size = group_size
        group0 = np.array(list(zip(list(np.random.normal(x0, var0, group_size)), list(np.random.normal(y0, var0, group_size)),[-1]*group_size)))
        group1 = np.array(list(zip(list(np.random.normal(x1, var1, group_size)), list(np.random.normal(y1, var0, group_size)),[1]*group_size)))

        self.data = np.concatenate((group0, group1))

        url = "https://i-learn-ml.oa.r.appspot.com/viewer/viewerSVM.html#"
        #grid = 30
        self.disp_url = url+f"xmax={grid_x}&xmin=-{grid_x}&ymax={grid_y}&ymin=-{grid_y}&"
        
        url2 = "https://i-learn-ml.oa.r.appspot.com/viewer/viewerGD.html#"
        self.disp_url2 = url2+f"xmax={grid_x}&xmin=-{grid_x}&ymax={grid_y}&ymin=-{grid_y}&"

        #self.X, self.Y = np.concatenate((self.group0[:,0],self.group1[:,0])), np.concatenate((self.group0[:,1],self.group1[:,1]))

  def calc(self, x, y): 
    '''act0 = np.tanh(x*self.group0[:,0]+y*self.group0[:,1])+1
    act1 = 1-np.tanh(x*self.group1[:,0]+y*self.group1[:,1])

    loss = (np.sum(act0)+np.sum(act1))/(2*self.group_size)'''
    pred = np.sign(np.tanh(x*self.data[:,0]+y*self.data[:,1]))
    #print(pred, self.data[:,2])
    acc = np.sum(np.equal(pred, self.data[:,2]))/(2*self.group_size)
    #print(acc)
    #loss = np.mean(1-np.tanh(self.data[:,2]*(x*self.data[:,0]+y*self.data[:,1])))
    '''self.scores = (x*self.data[:,0]+y*self.data[:,1])
    self.keep = np.exp(-self.data[:,2]*self.scores)/(1+np.exp(-self.data[:,2]*self.scores))
    loss = np.mean(self.keep)'''
    loss = np.mean(-self.data[:,2]*(x*self.data[:,0]+y*self.data[:,1]))
    return loss
  
  def pred(self):
    pred_y = np.sign(np.tanh(self.x_cur*self.data[:,0]+self.y_cur*self.data[:,1]))
    acc = np.sum(np.equal(pred_y, self.data[:,2]))/(2*self.group_size)
    return acc
  
  def derive(self, x, y): 
    '''der0 = 1/(2*self.group_size*np.cosh(x*self.group0[:,0]+y*self.group0[:,1])**2)
    der1 = -1/(2*self.group_size*np.cosh(x*self.group1[:,0]+y*self.group1[:,1])**2)
    dx = np.sum(self.group0[:,0]*der0) + np.sum(self.group1[:,0]*der1)
    dy = np.sum(self.group0[:,1]*der0) + np.sum(self.group1[:,1]*der1)'''
    
    '''der = -1/np.cosh(self.data[:,2]*(x*self.data[:,0]+y*self.data[:,1]))**2
    dx = np.mean(self.data[:,2]*self.data[:,0]*der)
    dy = np.mean(self.data[:,2]*self.data[:,1]*der)'''
    
    '''der = -self.keep*(1-self.keep)
    dx = np.mean(self.data[:,2]*self.data[:,0]*der)
    dy = np.mean(self.data[:,2]*self.data[:,1]*der)'''

    dx = np.mean(-self.data[:,2]*self.data[:,0])
    dy = np.mean(-self.data[:,2]*self.data[:,1])

    return [dx, dy]

  def display(self): 
    points_str = ""
    x = 30
    for i in range(len(self.Xs)):
      points_str+=(str)(-x)+","+(str)(x*self.Xs[i]/self.Ys[i])+","+(str)(0)+"|"+(str)(x)+","+(str)(-x*self.Xs[i]/self.Ys[i])+","+(str)(0)+"|"
    points_str=points_str[:-1]

    points_red = ""
    for (x,y) in self.data[:self.group_size,:2]:
      points_red += f"{x},{y},0"+"|"
    points_red=points_red[:-1]
    
    points_blue = ""
    for (x,y) in self.data[self.group_size:,:2]:
      points_blue += f"{x},{y},0"+"|"
    points_blue=points_blue[:-1]

    src=self.disp_url+f"points_red={points_red}&points_blue={points_blue}&vec={points_str}"
    return src
  
  def display2(self): 
    points_str = ""
    for i in range(len(self.Xs)):
      points_str+=(str)(self.Xs[i])+","+(str)(self.Ys[i])+","+(str)(self.Zs[i])+"|"
    points_str=points_str[:-1]

    x_sum = np.mean(-self.data[:,2]*self.data[:,0])
    y_sum = np.mean(-self.data[:,2]*self.data[:,1])
    
    f_str = f"{x_sum}*x+{y_sum}*y".replace("+-","-")

    src=self.disp_url2+f"func={f_str}&points={points_str}"
    return src

In [ ]:
T = SVM(50, (-5,-5,5), (5,5,5), lr=0.5, epochs=30)
print(f"accuracy before training: {100*T.pred()}%")
Xs, Ys, Zs = T.train()
print(f"accuracy after training: {100*T.pred()}%")

src1 = T.display()

IFrame(src=src1, width=700, height=600)

accuracy before training: 87.0%
accuracy after training: 92.0%


In [ ]:
src2 = T.display2()

IFrame(src=src2, width=700, height=600)

# test-functions

In [50]:
e = exp(1)

class Gaussian_helper: 
    def __init__(self, a, px, py, c=1): 
        self.a=a
        self.px=px
        self.py=py
        self.c=c

    def calc(self, x, y): 
        return -self.a*e**(-((x-self.px)**2+(y-self.py)**2)/(2*self.c**2))

    def derive(self, x, y): 
        tmp = (self.a/self.c**2)*e**(-((x-self.px)**2+(y-self.py)**2)/(2*self.c**2))
        return (tmp*(x-self.px), tmp*(y-self.py))

class Gaussian(Trainer):

    def __init__(self, apc:list, grid_x, grid_y, lr=0.5, epochs=50, opt=SGD, x=None, y=None): 
        super().__init__(grid_x, grid_y, lr, epochs, opt, x, y)
        self.g_list = [Gaussian_helper(*i) for i in apc]

        url = "https://i-learn-ml.oa.r.appspot.com/viewer/viewerGD.html#"
        self.disp_url = url+f"xmax={grid_x}&xmin=-{grid_x}&ymax={grid_y}&ymin=-{grid_y}&"

        self.f_str = ""
        for a, px, py, c in apc:
            s_x = "-"+(str)(px) if px>0 else "+"+(str)(-px)
            s_y = "-"+(str)(py) if py>0 else "+"+(str)(-py)
            self.f_str += f"-{a}*e^(-((x{s_x})^2+(y{s_y})^2)/(2*{c}^2))"
        
    def calc(self, x, y): 
        return sum([G.calc(x, y) for G in self.g_list])

    def derive(self, x, y): 
        d_list = [G.derive(x, y) for G in self.g_list]
        return [sum(i) for i in zip(*d_list)]
    
    def display(self): 
        points_str = ""
        for i in range(len(self.Xs)):
          points_str+=(str)(self.Xs[i])+","+(str)(self.Ys[i])+","+(str)(self.Zs[i])+"|"
        points_str=points_str[:-1]

        src=self.disp_url+f"func={self.f_str}&points={points_str}"
        return src

In [51]:
class Rosenbrock(Trainer):

    def __init__(self, a=1, b=100, grid_x=3, grid_y=3, lr=0.0001, epochs=50, opt=SGD, x=None, y=None): 
        super().__init__(grid_x, grid_y, lr, epochs, opt, x, y)

        url = "https://i-learn-ml.oa.r.appspot.com/viewer/viewerGD.html#"
        self.disp_url = url+f"xmax={grid_x}&xmin=-{grid_x}&ymax={grid_y}&ymin=-{grid_y}&"

        self.a, self.b = a, b
        self.f_str = f"({a}-x)^2+{b}*(y-x^2)^2"
        
        
    def calc(self, x, y): 
        return (self.a-x)**2+self.b*(y-x**2)**2

    def derive(self, x, y): 
        dx = 2*x*(-2*self.b*(y-x**2)-1)
        dy = 2*self.b*(y-x**2)
        return [dx, dy]
    
    def derive2(self, x, y): 
        dxx = 12*self.b*x**2-4*self.b*y+2
        dxy = -4*self.b*x
        dyx = -4*self.b*x
        dyy = 2*self.b
        return [[dxx, dxy], [dyx, dyy]]
    
    def display(self): 
        points_str = ""
        for i in range(len(self.Xs)):
          points_str+=(str)(self.Xs[i])+","+(str)(self.Ys[i])+","+(str)(self.Zs[i])+"|"
        points_str=points_str[:-1]

        src=self.disp_url+f"func={self.f_str}&points={points_str}"
        return src

In [52]:
class Rastrigin(Trainer):

    def __init__(self, A=10, grid_x=6, grid_y=6, lr=0.0001, epochs=50, opt=SGD, x=None, y=None): 
        super().__init__(grid_x, grid_y, lr, epochs, opt, x, y)

        url = "https://i-learn-ml.oa.r.appspot.com/viewer/viewerGD.html#"
        self.disp_url = url+f"xmax={grid_x}&xmin=-{grid_x}&ymax={grid_y}&ymin=-{grid_y}&"

        self.A = A
        self.f_str = f"{2*A}+x^2-{A}*cos({2*pi}*x)+y^2-{A}*cos({2*pi}*y)"
        
        
    def calc(self, x, y): 
        return 2*self.A+x**2-self.A*math.cos(2*pi*x)+y**2-self.A*math.cos(2*pi*y)

    def derive(self, x, y): 
        dx = 2*(pi*self.A*math.sin(2*pi*x)+x)
        dy = 2*(pi*self.A*math.sin(2*pi*y)+y)
        return [dx, dy]
    
    def display(self): 
        points_str = ""
        for i in range(len(self.Xs)):
          points_str+=(str)(self.Xs[i])+","+(str)(self.Ys[i])+","+(str)(self.Zs[i])+"|"
        points_str=points_str[:-1]

        src=self.disp_url+f"func={self.f_str}&points={points_str}"
        return src

In [86]:
class Himmelblau(Trainer):

    def __init__(self, grid_x=6, grid_y=6, lr=0.001, epochs=50, opt=SGD, x=None, y=None): 
        super().__init__(grid_x, grid_y, lr, epochs, opt, x, y)

        url = "https://i-learn-ml.oa.r.appspot.com/viewer/viewerGD.html#"
        self.disp_url = url+f"xmax={grid_x}&xmin=-{grid_x}&ymax={grid_y}&ymin=-{grid_y}&"

        self.f_str = f"(x^2+y-11)^2+(x+y^2-7)^2"
        
        self.f_str_arr = []
        
        
    def calc(self, x, y): 
        return (x**2+y-11)**2+(x+y**2-7)**2

    def derive(self, x, y): 
        dx = 2*(2*x*(x**2+y-11)+x+y**2-7)
        dy = 2*(2*y*(x+y**2-7)+x**2+y-11)

        self.f_str_arr.append(self.calc_f())
        return [dx, dy]
    
    def derive2(self, x, y): 
        dxx = 12*x**2+4*y-42
        dxy = 4*(x+y)
        dyx = 4*(x+y)
        dyy = 12*y**2+4*x-26
        return [[dxx, dxy], [dyx, dyy]]
    
    def display(self): 
        points_str = ""
        for i in range(len(self.Xs)):
          points_str+=(str)(self.Xs[i])+","+(str)(self.Ys[i])+","+(str)(self.Zs[i])+"|"
        points_str=points_str[:-1]

        src=self.disp_url+f"func={self.f_str}&points={points_str}"
        return src
    
    def calc_f(self): 

        '''dx = f"2*(2*{self.x_cur}*({self.x_cur}^2+{self.y_cur}-11)+{self.x_cur}+{self.y_cur}^2-7)"
        dy = f"2*(2*{self.y_cur}*({self.x_cur}+{self.y_cur}^2-7)+{self.x_cur}^2+{self.y_cur}-11)"
        dxx = f"(12*{self.x_cur}^2+4*{self.y_cur}-42)"
        dxy = f"4*({self.x_cur}+{self.y_cur})"
        dyx = f"4*({self.x_cur}+{self.y_cur})"
        dyy = f"(12*{self.y_cur}^2+4*{self.x_cur}-26)"
        f_str = f"({self.x_cur}^2+{self.y_cur}-11)^2+({self.x_cur}+{self.y_cur}^2-7)^2"'''

        x, y = self.x_cur, self.y_cur
        d = self.derive2(self.x_cur, self.y_cur)

        dx = 2*(2*x*(x**2+y-11)+x+y**2-7)
        dy = 2*(2*y*(x+y**2-7)+x**2+y-11)
        dxx, dxy = d[0]
        dyx, dyy = d[1]
        f_val = self.calc(self.x_cur, self.y_cur)
        
        Hx = f"((x-{self.x_cur})*{dxx}+(y-{self.y_cur})*{dyx})"
        Hy = f"((x-{self.x_cur})*{dxy}+(y-{self.y_cur})*{dyy})"

        A = f"0.5*(x-{self.x_cur})*{Hx}+(y-{self.y_cur})*{Hy}"

        B = f"((x-{self.x_cur})*{dx}+(y-{self.y_cur})*{dy})"

        f_str2 = f"{f_val}+{A}+{B}"

        src=f_str2
        return src

    def display2(self): 
        points_str = ""
        for i in range(len(self.Xs)):
          points_str+=(str)(self.Xs[i])+","+(str)(self.Ys[i])+","+(str)(self.Zs[i])+"|"
        points_str=points_str[:-1]

        src=self.disp_url+f"func={self.f_str}&points={points_str}&curves="
        for func in self.f_str_arr:
          src += func + "|"
        
        src = src[:-1]
        
        return src
      

In [54]:
class Beale(Trainer):

    def __init__(self, grid_x=4, grid_y=4, lr=0.0001, epochs=50, opt=SGD, x=None, y=None): 
        super().__init__(grid_x, grid_y, lr, epochs, opt, x, y)

        url = "https://i-learn-ml.oa.r.appspot.com/viewer/viewerGD.html#"
        self.disp_url = url+f"xmax={grid_x}&xmin=-{grid_x}&ymax={grid_y}&ymin=-{grid_y}&"

        self.f_str = f"(1.5-x+x*y)^2+(2.25-x+x*y^2)^2+(2.625-x+x*y^3)^2"
        
        
    def calc(self, x, y): 
        return (1.5-x+x*y)**2+(2.25-x+x*y**2)**2+(2.625-x+x*y**3)**2

    def derive(self, x, y): 
        dx = 2 * x *(y**6 + y**4 - 2 * y**3 - y**2 - 2 * y + 3) + 5.25 * y**3 + 4.5 * y**2 + 3 * y - 12.75
        dy = 6 * x * (x * (y**5 + 0.666667 * y**3 - y**2 - 0.333333 * y - 0.333333) + 2.625 * y**2 + 1.5 * y + 0.5)
        return [dx, dy]
    
    def display(self): 
        points_str = ""
        for i in range(len(self.Xs)):
          points_str+=(str)(self.Xs[i])+","+(str)(self.Ys[i])+","+(str)(self.Zs[i])+"|"
        points_str=points_str[:-1]

        src=self.disp_url+f"func={self.f_str}&points={points_str}"
        return src

In [55]:
class Absfunc(Trainer):

    def __init__(self, grid_x=10, grid_y=10, lr=0.0001, epochs=50, opt=SGD, x=None, y=None): 
        super().__init__(grid_x, grid_y, lr, epochs, opt, x, y)

        url = "https://i-learn-ml.oa.r.appspot.com/viewer/viewerGD.html#"
        self.disp_url = url+f"xmax={grid_x}&xmin=-{grid_x}&ymax={grid_y}&ymin=-{grid_y}&"

        #self.f_str = "x^4+y^4+50*sin(30*(x^2+y^2)*atan(y/x))"
        '''t = "3*atan(y/x)"
        self.f_str = f"x^2+y^2+20*abs({t}-floor({t})-0.5)"'''
        self.f_str = "abs(x)+abs(y)"
        
        
    def calc(self, x, y): 
        #return x**4+y**4+50*math.sin(30*(x**2+y**2)*math.atan(y/x))
        #return x**2+y**2+20*abs(3*math.atan(y/x)-math.floor(3*math.atan(y/x))-0.5)
        return abs(x)+abs(y)

    def derive(self, x, y): 
        #dx = (4 * (x**5 + x**3 * y**2 - 625 * y * math.cos(50 * math.atan(y/x))))/(x**2 + y**2)
        #dx = 4 * (x**3 + 750 * x * math.atan(y/x) * math.cos(30 * (x**2 + y**2) * math.atan(y/x)) - 375 * y * math.cos(30 * (x**2 + y**2) * math.atan(y/x)))
        #dy = 4 * ((625 * x * math.cos(50 * math.atan(y/x)))/(x**2 + y**2) + y**3)
        #dy = 4 * (750 * y * math.atan(y/x) * math.cos(30 * (x**2 + y**2) * math.atan(y/x)) + 375 * x * math.cos(30 * (x**2 + y**2) * math.atan(y/x)) + y**3)
        
        '''if math.sin(3 * pi * math.atan(y/x))!=0 and 3 * math.atan(y/x) - math.floor(3 * math.atan(y/x))>0.5:
          dx = 2 * x - (60 * y)/(x**2 + y**2)
          dy = 2 * y + (60 * x)/(x**2 + y**2)
        else:
          dx = 2 * x +  (60 * y)/(x**2 + y**2)
          dy = 2 * y - (60 * x)/(x**2 + y**2)'''
        
        dx = x/abs(x) if x!=0 else 0
        dy = y/abs(y) if y!=0 else 0

        return [dx, dy]
    
    def display(self): 
        points_str = ""
        for i in range(len(self.Xs)):
          points_str+=(str)(self.Xs[i])+","+(str)(self.Ys[i])+","+(str)(self.Zs[i])+"|"
        points_str=points_str[:-1]

        src=self.disp_url+f"func={self.f_str}&points={points_str}"
        return src
    

In [56]:
sizes = []
for ix in [-15,0,15]:
  for iy in [-15,0,15]:
    sizes.append((50,ix,iy,5))

grid_x = 30
grid_y = 30

G = Gaussian(sizes, grid_x, grid_y)

Xs, Ys, Zs = G.train()

src = G.display()

IFrame(src=src, width=700, height=600)

In [57]:
Ro = Rosenbrock(lr=0.1**5, opt=SGD, grid_x=10, grid_y=10, epochs=100, x=5, y=-5)

Xs, Ys, Zs = Ro.train()

src = Ro.display()

IFrame(src=src, width=700, height=600)

In [58]:
Ra = Rastrigin()

Xs, Ys, Zs = Ra.train()

src = Ra.display()

IFrame(src=src, width=700, height=600)

In [98]:
Hi = Himmelblau(opt=Newton, lr=0.25, x=5, y=5, epochs=40)

Xs, Ys, Zs = Hi.train()

src = Hi.display()

IFrame(src=src, width=700, height=600)

In [100]:
src2 = Hi.display2()

IFrame(src=src2, width=700, height=600)

In [62]:
Be = Beale(opt=MomentumSGD, lr=0.000001, x=4, y=4)

Xs, Ys, Zs = Be.train()

src = Be.display()

IFrame(src=src, width=700, height=600)

In [82]:
Abs = Absfunc(opt=MomentumSGD, lr=0.1, x=10, y=0.5)

Xs, Ys, Zs = Abs.train()

src = Abs.display()

IFrame(src=src, width=700, height=600)